In [1]:
!pip install diffusers transformers accelerate scipy safetensors accelerate streamlit googletrans==3.1.0a0 -q
!npm install -g localtunnel -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 133.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [6]:
%%writefile app.py

import streamlit as st
import torch
from googletrans import Translator
from diffusers import StableDiffusionPipeline, DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
import subprocess
import os

# Cache the image model loading function
@st.cache_resource
def load_image_model():
    model_id = "stabilityai/stable-diffusion-2"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    return model.to(device)

# Cache the video model loading function
@st.cache_resource
def load_video_model():
    model_id = "damo-vilab/text-to-video-ms-1.7b"
    model = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
    model.scheduler = DPMSolverMultistepScheduler.from_config(model.scheduler.config)
    model.enable_model_cpu_offload()
    return model

# Load models
image_model = load_image_model()
video_model = load_video_model()

# Translate other language input to English
def get_translation(text, dest_lang='en'):
    translator = Translator()
    translated_text = translator.translate(text, dest=dest_lang)
    return translated_text.text

# Returns generated image
def image_generator(prompt):
    with torch.no_grad():
        image = image_model(prompt).images[0]
    return image

# Returns path to generated video
def video_generator(prompt, length=5):
    video_frames = video_model(
        prompt=prompt,
        num_frames=length*5,
        num_inference_steps=10,
        guidance_scale=6.5
    ).frames
    frame_rate = 5  # Frames per second (FPS) for the video
    video_path = export_to_video(video_frames[0], fps=frame_rate)
    return video_path

# Main Page
def main_page():
    st.title("Media Generator")

    # Create two columns for layout
    col1, col2 = st.columns(2)

    with col1:
        st.header("Input")
        text_input = get_translation(st.text_input("Enter some text:", key="text_input"),'en')
        media_type = st.radio("Select media type:", ("Image", "Video"))
        if(media_type == 'Video'):
          video_length = st.number_input("Enter the length of the video (seconds):", min_value=1, step=1)

        if st.button("Generate"):
            st.session_state["current_prompt"] = text_input

            # Clear previous generation results
            if "generated_image" in st.session_state:
                del st.session_state["generated_image"]
            if "generated_video" in st.session_state:
                del st.session_state["generated_video"]

            # Generate new content based on the selected media type
            if media_type == "Image":
                st.session_state["generated_image"] = image_generator(text_input)
                st.session_state["generated_media_type"] = "Image"
            else:
                st.session_state["generated_video"] = video_generator(text_input, length=video_length)
                st.session_state["generated_media_type"] = "Video"

            st.session_state["last_prompt"] = text_input

    with col2:
        st.header("Output")
        if "generated_media_type" in st.session_state:
            if st.session_state["generated_media_type"] == "Image":
                image = st.session_state["generated_image"]
                st.image(image, caption="Generated Image")
                st.download_button(
                    label="Download Image",
                    data=image.tobytes(),  # Convert the image to bytes
                    file_name="generated_image.png",
                    mime="image/png"
                )
            else:
                video_path = st.session_state["generated_video"]

                # converting mp4 to h264
                input_file = video_path
                output_file = '/content/output.mp4'

                if(os.path.exists(output_file)):
                    os.remove(output_file)

                command = [
                    'ffmpeg',
                    '-i', input_file,
                    '-c:v', 'libx264',
                    '-crf', '23',
                    '-preset', 'medium',
                    output_file
                ]

                subprocess.run(command, check=True)

                st.video(open(output_file, "rb").read())
                st.download_button(
                    label="Download Video",
                    data=open(output_file, "rb").read(),  # Read the video file
                    file_name="generated_video.mp4",
                    mime="video/mp4"
                )

if __name__ == "__main__":
    main_page()

Overwriting app.py


In [ ]:
'''
after starting the server then click on the link 'your url is: ' then a window would open up asking for a
ip address then just paste the ip address in the external link there and move forward.

id addess format -  num1.num2.num3.num4

number after : is port number and not part of ip address
'''


!streamlit run app.py &  npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.87.140.62:8501

your url is: https://fruity-aliens-march.loca.lt
2024-08-02 15:14:26.547491: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 15:14:26.734483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 15:14:26.764213: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 15:14:28.401718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading pipeline 